<a href="https://colab.research.google.com/github/mfarrokhrouz/bigdata/blob/main/practising_partitions_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main Sources [2,578 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder\
    .appName("SparkSQL")\
    .config("spark.sql.debug.maxToStringFields", 2000)\
    .config("spark.driver.memory", "2g")\
    .getOrCreate()

In [3]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/3/DelayedFlights.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("DelayedFlights.csv"), sep=",", header=True)
df.show()

+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
| id|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|  0|2008|    1|         3|        4|   2003|      1955|   2211|      2225|       

In [8]:
# Create temp view named "delays"
df.createOrReplaceTempView('delays')

# Start the runtime
start_time = time.time()

# Using spark.sql write a query that gives you the total distance 
# and the count of every unique Origin, Dest combination.
spark.sql("""select Origin,Dest,sum(Distance),count(Origin) from delays group by Origin,Dest""").show()


# Print out the runtime.
print("--- %s seconds ---" % (time.time() - start_time))

+------+----+-------------+-------------+
|Origin|Dest|sum(Distance)|count(Origin)|
+------+----+-------------+-------------+
|   LAS| LIT|      72520.0|           56|
|   PHL| MCO|     986706.0|         1146|
|   SMF| BUR|     178284.0|          498|
|   SNA| PHX|     218010.0|          645|
|   MCI| IAH|     156249.0|          243|
|   BFL| SAN|       4515.0|           21|
|   ROC| CLE|      39935.0|          163|
|   SPI| ORD|      34104.0|          196|
|   ATL| GSP|      54621.0|          357|
|   SFO| PMD|      26860.0|           85|
|   LAX| OXR|       6958.0|          142|
|   ORD| PDX|     909497.0|          523|
|   PBI| DCA|      99412.0|          116|
|   FSD| ATL|      56286.0|           59|
|   MLI| MCO|      25900.0|           25|
|   MSP| AVL|      24969.0|           29|
|   BQN| MCO|      48547.0|           43|
|   EWR| STT|      73530.0|           45|
|   CLE| SJU|       1839.0|            1|
|   MCI| MKE|        436.0|            1|
+------+----+-------------+-------

In [9]:
# Write out the data in parquet format
df.write.parquet('parquet_delayed', mode='overwrite')

In [10]:
# Read in our new parquet formatted data
p_df=spark.read.parquet('parquet_delayed')

In [11]:
# Convert the DataFrame to a view.
p_df.createOrReplaceTempView('p_delays')

In [12]:
# Start the runtime
start_time = time.time()

# Run the same query here
spark.sql("""select Origin,Dest,sum(Distance),count(Origin) from p_delays group by Origin,Dest""").show()


# Print out the runtime
print("--- %s seconds ---" % (time.time() - start_time))

+------+----+-------------+-------------+
|Origin|Dest|sum(Distance)|count(Origin)|
+------+----+-------------+-------------+
|   LAS| LIT|      72520.0|           56|
|   PHL| MCO|     986706.0|         1146|
|   SMF| BUR|     178284.0|          498|
|   SNA| PHX|     218010.0|          645|
|   MCI| IAH|     156249.0|          243|
|   BFL| SAN|       4515.0|           21|
|   ROC| CLE|      39935.0|          163|
|   SPI| ORD|      34104.0|          196|
|   ATL| GSP|      54621.0|          357|
|   SFO| PMD|      26860.0|           85|
|   LAX| OXR|       6958.0|          142|
|   ORD| PDX|     909497.0|          523|
|   PBI| DCA|      99412.0|          116|
|   FSD| ATL|      56286.0|           59|
|   MLI| MCO|      25900.0|           25|
|   MSP| AVL|      24969.0|           29|
|   BQN| MCO|      48547.0|           43|
|   EWR| STT|      73530.0|           45|
|   CLE| SJU|       1839.0|            1|
|   MCI| MKE|        436.0|            1|
+------+----+-------------+-------

In [14]:
# Write out your parquet data, partitioning on the Origin column
df.write.partitionBy("UniqueCarrier").mode("overwrite").parquet("delayed_partitioned")

In [15]:
# Read in our new parquet formatted data
p_df_p=spark.read.parquet('delayed_partitioned')

In [16]:
# Convert the dataframe to a view.
p_df_p.createOrReplaceTempView('p_delays_p')

In [18]:
# Start the runtime
start_time = time.time()

# Run your query against your partitioned data one more time.
spark.sql("""select Origin,sum(Distance),count(Origin) from p_delays_p group by Origin""").show()


# Print out the runtime
print("--- %s seconds ---" % (time.time() - start_time))

+------+-------------+-------------+
|Origin|sum(Distance)|count(Origin)|
+------+-------------+-------------+
|   MSY|    3862683.0|         5979|
|   GEG|    1061773.0|         1913|
|   SNA|    4862882.0|         6222|
|   BUR|    1985928.0|         4081|
|   GTF|     101730.0|          154|
|   GRB|     236690.0|         1095|
|   IDA|     122755.0|          296|
|   GRR|     762389.0|         2082|
|   EUG|     274636.0|          522|
|   PVD|    2055464.0|         3147|
|   GSO|     868253.0|         1714|
|   MYR|     188781.0|          505|
|   OAK|    6514328.0|         9941|
|   MSN|     624663.0|         1839|
|   FAR|     253367.0|          545|
|   COD|      17220.0|           54|
|   BTM|       8925.0|           25|
|   FSM|      85950.0|          269|
|   MQT|      30865.0|          134|
|   DCA|    5820315.0|         9151|
+------+-------------+-------------+
only showing top 20 rows

--- 1.627490758895874 seconds ---


In [20]:
# Start  the runtime
start_time = time.time()

# Filter the data on something that selects your partition choice.
spark.sql("""select Origin,TailNum from p_delays_p group by Origin,TailNum""").show()


# Print out the runtime.
print("--- %s seconds ---" % (time.time() - start_time))

+------+-------+
|Origin|TailNum|
+------+-------+
|   LAS| N678AA|
|   LAS| N429WN|
|   OAK| N655WN|
|   PHX| N709SW|
|   PVD| N228WN|
|   SLC| N756SA|
|   BNA| N228WN|
|   BUR| N718SW|
|   HOU| N525SW|
|   HOU| N438WN|
|   LAS| N486WN|
|   LAX| N253WN|
|   MCO| N710SW|
|   ONT| N344SW|
|   SEA| N487WN|
|   SJC| N368SW|
|   ALB| N430WN|
|   BDL| N765SW|
|   BHM| N324SW|
|   BUR| N391SW|
+------+-------+
only showing top 20 rows

--- 2.6046736240386963 seconds ---


In [21]:
# Start  the runtime
start_time = time.time()

# Filter the data on something that has nothing to do with your partition choice.
spark.sql("""select Dest,TailNum from p_delays group by Dest,TailNum""").show()


# Print out the runtime.
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------+
|Dest|TailNum|
+----+-------+
| SLC| N756SA|
| LAS| N678AA|
| BUR| N718SW|
| DTW| N412WN|
| LAS| N429WN|
| BUR| N384SW|
| ONT| N344SW|
| PHL| N699SW|
| RDU| N688SW|
| DEN| N717SA|
| HOU| N438WN|
| SJC| N368SW|
| LAX| N253WN|
| LAS| N486WN|
| SEA| N487WN|
| LAS| N514SW|
| FLL| N310SW|
| HOU| N784SW|
| BUR| N391SW|
| PBI| N771SA|
+----+-------+
only showing top 20 rows

--- 1.4443652629852295 seconds ---
